In [1]:
from mne.io import concatenate_raws, read_raw_edf
import mne
import glob
import numpy as np
import xlrd
import pandas as pd

In [2]:
path = "../SE_parameter/batch1_stimulate_parameter.xlsx"
df = pd.read_excel(path,engine='openpyxl')
#df= df.astype({'完成时间':'object'})
#对df进行排序，为了和glob.glob的顺序相匹配
df=df.sort_values(by=["姓名","完成时间"])

In [3]:
df=df.reset_index()
df=df.drop(['index','Unnamed: 0'],axis=1)

In [4]:
df.shape

(96, 5)

In [5]:
df

,姓名,性别,年龄,刺激参数,完成时间
0,何富桂,女,53,1,20220221
1,何富桂,女,53,4,20220225
2,何富桂,女,53,3,20220228
3,何富桂,女,53,2,20220302
4,倪诗瑄,女,16,3,20220311
...,...,...,...,...,...
91,黄金燕,女,24,2,20220316
92,齐金钢,男,27,1,20220223
93,齐金钢,男,27,4,20220225
94,齐金钢,男,27,2,20220228


In [6]:
ECT_code = []

In [7]:
for i in range(df.shape[0]):
    ECT_code.append(df.iloc[i,3])

In [8]:
ECT_code = np.array(ECT_code)

In [9]:
ECT_code

array([1, 4, 3, 2, 3, 4, 2, 1, 4, 2, 1, 3, 4, 2, 3, 1, 2, 4, 1, 3, 4, 3,
       1, 2, 3, 2, 1, 4, 2, 3, 1, 4, 3, 1, 2, 4, 2, 3, 4, 1, 3, 1, 4, 2,
       4, 1, 2, 3, 1, 3, 2, 4, 2, 1, 3, 4, 4, 1, 2, 3, 4, 3, 2, 1, 2, 4,
       3, 1, 1, 2, 3, 4, 3, 2, 4, 1, 2, 1, 4, 3, 3, 4, 1, 2, 1, 2, 4, 3,
       1, 3, 4, 2, 1, 4, 2, 3], dtype=int64)

In [11]:
# rest_path = r"../patient_data/静息脑电（第一批次）/"
therapy_path=r"../patient_data/治疗脑电（第一批次）/"

# rest_subdirectory = sorted(glob.glob(rest_path+"*"))
# therapy_subdirectory = sorted(glob.glob(therapy_path+"*"))
therapy_file_path = sorted(glob.glob(therapy_path+"*"))

In [12]:
len(therapy_file_path)

96

In [13]:
path_parameter_dict={}

In [14]:
#构建一个 {路径:ECT参数} 的字典，方便打乱顺序后，仍能保持一一对应
for i in range(len(therapy_file_path)):
    path_parameter_dict[therapy_file_path[i]]=ECT_code[i]

In [15]:
path_parameter_dict

{'../patient_data/治疗脑电（第一批次）\\Z何富桂20220221.edf': 1,
 '../patient_data/治疗脑电（第一批次）\\Z何富桂20220225.edf': 4,
 '../patient_data/治疗脑电（第一批次）\\Z何富桂20220228.edf': 3,
 '../patient_data/治疗脑电（第一批次）\\Z何富桂20220302.edf': 2,
 '../patient_data/治疗脑电（第一批次）\\Z倪诗瑄20220311.edf': 3,
 '../patient_data/治疗脑电（第一批次）\\Z倪诗瑄20220314.edf': 4,
 '../patient_data/治疗脑电（第一批次）\\Z倪诗瑄20220316.edf': 2,
 '../patient_data/治疗脑电（第一批次）\\Z倪诗瑄20220318.edf': 1,
 '../patient_data/治疗脑电（第一批次）\\Z刘世清20220309.edf': 4,
 '../patient_data/治疗脑电（第一批次）\\Z刘世清20220311.edf': 2,
 '../patient_data/治疗脑电（第一批次）\\Z刘世清20220314（未发作）.edf': 1,
 '../patient_data/治疗脑电（第一批次）\\Z刘世清20220316.edf': 3,
 '../patient_data/治疗脑电（第一批次）\\Z刘家瑞20220225.edf': 4,
 '../patient_data/治疗脑电（第一批次）\\Z刘家瑞20220228.edf': 2,
 '../patient_data/治疗脑电（第一批次）\\Z刘家瑞20220302.edf': 3,
 '../patient_data/治疗脑电（第一批次）\\Z刘家瑞20220304.edf': 1,
 '../patient_data/治疗脑电（第一批次）\\Z周小凯20220314（未发作）.edf': 2,
 '../patient_data/治疗脑电（第一批次）\\Z周小凯20220316.edf': 4,
 '../patient_data/治疗脑电（第一批次）\\Z周小凯20220318.edf': 1,
 '

In [16]:
#观察文件名，发现所有文件名均为“Z人名日期”，所以取文件名的[1:4]为人名，前后对照，如果不同则表示是不同的人
#思路：首先将所有人名取出，然后随机打乱顺序，前20个人作为训练数据，后4个人作为测试数据
therapy_file_path[0].replace('\\','/').split('/')[-1][1:4]

'何富桂'

In [17]:
#获得名单
def getNameList(file_path,name_list):
    last_name='1'
    for i in file_path:
        cur_name=i.replace('\\','/').split('/')[-1][1:4]
        if (last_name!=cur_name):
            name_list.append(cur_name)
        last_name=cur_name      

In [28]:
#打乱名单，体现随机性
np.random.seed(42)
name_list=[]
getNameList(therapy_file_path,name_list)
np.random.shuffle(name_list)
name_list

['徐珍英',
 '王纯刚',
 '何富桂',
 '肖智华',
 '杜娇2',
 '曾刚2',
 '林攀攀',
 '倪诗瑄',
 '黄盈雪',
 '孙彭辉',
 '刘世清',
 '林儒波',
 '王宜凡',
 '刘家瑞',
 '周小凯',
 '黄金燕',
 '肖星星',
 '雷振2',
 '齐金钢',
 '张璐2',
 '李玉萍',
 '沈伟2',
 '赵丽兰',
 '宋志辉']

In [29]:
len(name_list)

24

In [30]:
#此处设定有多少患者用于训练集(人数)
train_patient_num=20
total_patient_num=len(name_list)
train_therapy_file_path=[]
test_therapy_file_path=[]
for i in range(train_patient_num):
    temp_file_path=sorted(glob.glob(therapy_path+'*'+name_list[i]+'*'))
    for j in temp_file_path:
        train_therapy_file_path.append(j)
for i in range(train_patient_num,total_patient_num):
    temp_file_path=sorted(glob.glob(therapy_path+'*'+name_list[i]+'*'))
    for j in temp_file_path:
        test_therapy_file_path.append(j)

In [31]:
#这里必须要排序
train_therapy_file_path.sort()
test_therapy_file_path.sort()

In [32]:
#统计未发作和发作的样本数
trigger_num=0
non_trigger_num=0
for i in train_therapy_file_path:
    # 找不到未发作，即发作
    if i.find('未发作')==-1:
        trigger_num+=1
    else:
        non_trigger_num+=1
print(trigger_num/non_trigger_num)

multiple=trigger_num/non_trigger_num

4.0


In [33]:
multiple = round(multiple)
multiple

4

In [34]:
non_trigger_num

16

In [35]:
trigger_num+non_trigger_num*multiple

128

In [36]:
#抖动
def jitter(data):
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            #抖动因子
            jitter_factor=np.random.randint(1,11)
            #抖动方向
            jitter_direction=np.random.randint(1,3)
            #变化在 *0.9 至 *1.1内
            if jitter_direction==1:
                data[i][j]= data[i][j]*(1+jitter_factor/100)
            else:
                data[i][j]= data[i][j]*(1-jitter_factor/100)

In [37]:
#翻转
def flip(data):
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            data[i][j]=-data[i][j]

In [38]:
#随机通道翻转
def random_flip(data):
    flip_channel=np.random.randint(0,15)
    for i in range (data.shape[1]):
        data[flip_channel][i]=-data[flip_channel][i]

In [39]:
#扩大/压缩
def scale_up(data):
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            data[i][j]= data[i][j]*1.1
def scale_down(data):
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            data[i][j]= data[i][j]*0.9

In [40]:
#排列
def permutation(data,p_strategy):
    for i in range(0,16,2):
        #把2s片段分为4组,假设序号为1,2,3,4
        p1_start=int(i*1024)
        p1_end=int((i+0.5)*1024)
        p2_start=int((i+0.5)*1024)
        p2_end=int((i+1)*1024)
        p3_start=int((i+1)*1024)
        p3_end=int((i+1.5)*1024)
        p4_start=int((i+1.5)*1024)
        p4_end=int((i+2)*1024)
        for j in range(15):
            #排列方案为1时， 1,2,3,4-->2,3,4,1
            if p_strategy==1: 
                data_temp=[]
                data_temp.append(data[j][p2_start:p2_end])
                data_temp.append(data[j][p3_start:p3_end])
                data_temp.append(data[j][p4_start:p4_end])
                data_temp.append(data[j][p1_start:p1_end])
                data_temp=np.array(data_temp)
                data_temp=data_temp.reshape(1,-1)
                data_temp=data_temp[0]
                data=np.array(data)
                data[j,p1_start:p4_end]=data_temp
            #排列方案为2时， 1,2,3,4-->3,4,1,2
            elif p_strategy==2:
                data_temp=[]
                data_temp.append(data[j][p3_start:p3_end])
                data_temp.append(data[j][p4_start:p4_end])
                data_temp.append(data[j][p1_start:p1_end])
                data_temp.append(data[j][p2_start:p2_end])
                data_temp=np.array(data_temp)
                data_temp=data_temp.reshape(1,-1)
                data_temp=data_temp[0]
                data[j,p1_start:p4_end]=data_temp
            #排列方案为3时， 1,2,3,4-->4,1,2,3
            elif p_strategy==3:
                data_temp=[]
                data_temp.append(data[j][p4_start:p4_end])
                data_temp.append(data[j][p1_start:p1_end])
                data_temp.append(data[j][p2_start:p2_end])
                data_temp.append(data[j][p3_start:p3_end])
                data_temp=np.array(data_temp)
                data_temp=data_temp.reshape(1,-1)
                data_temp=data_temp[0]
                data[j,p1_start:p4_end]=data_temp

In [41]:
#检测EEG文件是否有flag标记,返回true/false
def judgeflag(file_path,flag):
    raw = read_raw_edf(file_path,preload=False)
    events_from_annot, event_dict = mne.events_from_annotations(raw)
    if flag in event_dict:
        return True
    else:
        return False

In [42]:
# raw=read_raw_edf(train_therapy_file_path[0],preload=False)

In [43]:
# data,times=raw[:,0:1024*16]
# raw2=mne.io.RawArray(data,raw.info)

In [44]:
# 数据采样率1024Hz,文件路径；采集时间；是否为测试集标志;数据增强策略;未发作的第几次随机通道翻转
def get_se_epochs(path,select_time,is_test,DA_strategy,random_flip_no):
    #选择15个通道
    chs1 = ['EEG Fp1-AV', 'EEG Fp2-AV', 'EEG F3-AV', 'EEG F4-AV', 'EEG C3-AV', 'EEG C4-AV', 'EEG P3-AV', 'EEG P4-AV',
       'EEG O1-AV', 'EEG O2-AV', 'EEG T3-AV', 'EEG T4-AV', 'EEG T5-AV', 'EEG T6-AV','EEG Cz-AV']
    chs2 =['EEG Fp1-A1', 'EEG Fp2-A2', 'EEG F3-A1','EEG F4-A2','EEG C3-A1','EEG C4-A2','EEG P3-A1','EEG P4-A2','EEG O1-A1',
           'EEG O2-A2','EEG T3-A1','EEG T4-A2','EEG T5-A1','EEG T6-A2','EEG Cz-AAV']
    # select_time 采集的时间
    raw=read_raw_edf(path,preload=False)
    print(raw.info.ch_names)
    #第一批数据采样为1024Hz，第二批变成了512Hz
    if chs1[0] in raw.info.ch_names:
        chs=chs1  
        freq=1024
    else:
        chs=chs2
        freq=512
    raw=raw.pick_channels(chs)
    events_from_annot, event_dict = mne.events_from_annotations(raw)
    # 找到SE，麻醉标记,save函数的参数单位是s，所以要除以1024
    start_time=events_from_annot[event_dict['SE']-2][0]
    end_time=start_time+select_time*freq
    print(start_time,end_time)
    #选择所有的15通道
    data,times = raw[:,start_time:end_time]
    print(times)
    #不做变化
    if (DA_strategy==0): 
        path_name_suffix='(original)'
    #抖动
    elif (DA_strategy==1):
        jitter(data)
        path_name_suffix='(jitter)'
    #翻转
    elif (DA_strategy==2):
        flip(data)
        path_name_suffix='(flip)'
    #扩大
    elif (DA_strategy==3):
        scale_up(data)
        path_name_suffix='(scale_up)'  
    #缩小
    elif (DA_strategy==4):
        scale_down(data)
        path_name_suffix='(scale_down)'  
    #排列1
    elif (DA_strategy==5):
        permutation(data,1)
        path_name_suffix='(permutation_1)'  
    #排列2
    elif (DA_strategy==6):
        permutation(data,2)
        path_name_suffix='(permutation_2)'  
    #排列3
    elif (DA_strategy==7):
        permutation(data,3)
        path_name_suffix='(permutation_3)'  
    #随机通道翻转
    elif (DA_strategy==8):
        random_flip(data)
        path_name_suffix='random_flip_'+str(random_flip_no)
    raw_temp=mne.io.RawArray(data,raw.info)
    path_name=path.split('\\')[1:]
    path_name="".join(path_name)
    path_name=path_name.split('.')[0]
    #在这里将训练集和测试集分开,需要新建SE_train和SE_test两个文件夹
    if (is_test==False):
        path_name=r"../SE_train/" + path_name +path_name_suffix+".fif"
    else:   
        path_name=r"../SE_test/" + path_name + ".fif"     
    path_name=path_name.replace('/','\\')
    print(path_name)
    raw_temp.save(path_name,overwrite=True)

In [45]:
file_without_se = []
df_parameter_train=pd.DataFrame(columns=['parameter'])
df_parameter_test=pd.DataFrame(columns=['parameter'])

In [46]:
#训练集SE切片长度
train_SE_len=16
#测试集SE切片长度
test_SE_len=16
#选择offset是多少的倍数
# offset_multiple=2

In [47]:
#生成SE_train
for file_path in train_therapy_file_path:
    if judgeflag(file_path,"SE"):
        #初始化random_flip_no，random_flip_no的作用是方便命名未发作文件，防止发生同名文件覆盖
        random_flip_no=0
        #如果发作，则只截取train_SE_len长度
        #为False时表示当前数据送入训练集
        #通过8种策略数据增强
        for DA_strategy in range(8):
            get_se_epochs(file_path,train_SE_len,False,DA_strategy,0)
            #添加参数到df_parameter_train的末尾
            df_parameter_train.loc[len(df_parameter_train.index)]=path_parameter_dict[file_path]
            #如果未发作，则随机通道翻转增加未发作样本数量
            if file_path.find('未发作')!=-1:
                for j in range(1,multiple):
                    #修改random_flip_no
                    random_flip_no+=1
                    #DA_strategy固定为8，即通过随机翻转达到0-1平衡
                    get_se_epochs(file_path,train_SE_len,False,8,random_flip_no) 
                    df_parameter_train.loc[len(df_parameter_train.index)]=path_parameter_dict[file_path]
    else:
        file_without_se.append(file_path)
#生成SE_test,不考虑对未发作数据的重叠采样
for file_path in test_therapy_file_path:
    if judgeflag(file_path,"SE"):
        #取test_SE_len片段
        #DA_strategy=0,即不做变化
        get_se_epochs(file_path,test_SE_len,True,0,0) 
        df_parameter_test.loc[len(df_parameter_test.index)]=path_parameter_dict[file_path]
    else:
        file_without_se.append(file_path)

Extracting EDF parameters from E:\jupyter file\EEG_process\patient_data\治疗脑电（第一批次）\Z何富桂20220221.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['File Create', 'Recording ends', 'Recording starts', 'SE', 'end', 'start']
Extracting EDF parameters from E:\jupyter file\EEG_process\patient_data\治疗脑电（第一批次）\Z何富桂20220221.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
['EEG Fp1-AV', 'EEG Fp2-AV', 'EEG F3-AV', 'EEG F4-AV', 'EEG C3-AV', 'EEG C4-AV', 'EEG P3-AV', 'EEG P4-AV', 'EEG O1-AV', 'EEG O2-AV', 'EEG T3-AV', 'EEG T4-AV', 'EEG T5-AV', 'EEG T6-AV', 'EEG A1-AV', 'EEG A2-AV', 'EEG Cz-AV', 'EEG ECG', 'EEG EMG']
Used Annotations descriptions: ['File Create', 'Recording ends', 'Recording starts', 'SE', 'end', 'start']
8919 25303
[ 8.70996094  8.7109375   8.71191406 ... 24.70703125 24.70800781
 24.70898438]
Creating RawArray with float64 data, n_channels=15, n_times=16384
    Range : 

<ipython-input-44-9e5188346da5>:73: RuntimeWarning: This filename (E:\jupyter file\EEG_process\SE_train\Z何富桂20220221(original).fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_temp.save(path_name,overwrite=True)


Creating RawArray with float64 data, n_channels=15, n_times=16384
    Range : 0 ... 16383 =      0.000 ...    15.999 secs
Ready.
..\SE_train\Z何富桂20220221(jitter).fif
Writing E:\jupyter file\EEG_process\SE_train\Z何富桂20220221(jitter).fif
Closing E:\jupyter file\EEG_process\SE_train\Z何富桂20220221(jitter).fif
[done]
Extracting EDF parameters from E:\jupyter file\EEG_process\patient_data\治疗脑电（第一批次）\Z何富桂20220221.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
['EEG Fp1-AV', 'EEG Fp2-AV', 'EEG F3-AV', 'EEG F4-AV', 'EEG C3-AV', 'EEG C4-AV', 'EEG P3-AV', 'EEG P4-AV', 'EEG O1-AV', 'EEG O2-AV', 'EEG T3-AV', 'EEG T4-AV', 'EEG T5-AV', 'EEG T6-AV', 'EEG A1-AV', 'EEG A2-AV', 'EEG Cz-AV', 'EEG ECG', 'EEG EMG']
Used Annotations descriptions: ['File Create', 'Recording ends', 'Recording starts', 'SE', 'end', 'start']
8919 25303
[ 8.70996094  8.7109375   8.71191406 ... 24.70703125 24.70800781
 24.70898438]
Creating RawArray with float64 data, n_channels=15, n_time

<ipython-input-44-9e5188346da5>:73: RuntimeWarning: This filename (E:\jupyter file\EEG_process\SE_train\Z何富桂20220221(jitter).fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_temp.save(path_name,overwrite=True)


    Range : 0 ... 16383 =      0.000 ...    15.999 secs
Ready.
..\SE_train\Z何富桂20220221(flip).fif
Writing E:\jupyter file\EEG_process\SE_train\Z何富桂20220221(flip).fif
Closing E:\jupyter file\EEG_process\SE_train\Z何富桂20220221(flip).fif
[done]
Extracting EDF parameters from E:\jupyter file\EEG_process\patient_data\治疗脑电（第一批次）\Z何富桂20220221.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
['EEG Fp1-AV', 'EEG Fp2-AV', 'EEG F3-AV', 'EEG F4-AV', 'EEG C3-AV', 'EEG C4-AV', 'EEG P3-AV', 'EEG P4-AV', 'EEG O1-AV', 'EEG O2-AV', 'EEG T3-AV', 'EEG T4-AV', 'EEG T5-AV', 'EEG T6-AV', 'EEG A1-AV', 'EEG A2-AV', 'EEG Cz-AV', 'EEG ECG', 'EEG EMG']
Used Annotations descriptions: ['File Create', 'Recording ends', 'Recording starts', 'SE', 'end', 'start']
8919 25303
[ 8.70996094  8.7109375   8.71191406 ... 24.70703125 24.70800781
 24.70898438]


...

<ipython-input-44-9e5188346da5>:73: RuntimeWarning: This filename (E:\jupyter file\EEG_process\SE_test\Z沈伟20220311.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_temp.save(path_name,overwrite=True)
<ipython-input-44-9e5188346da5>:73: RuntimeWarning: This filename (E:\jupyter file\EEG_process\SE_test\Z沈伟20220314（未发作）.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_temp.save(path_name,overwrite=True)
<ipython-input-44-9e5188346da5>:73: RuntimeWarning: This filename (E:\jupyter file\EEG_process\SE_test\Z沈伟20220316.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, r

Closing E:\jupyter file\EEG_process\SE_test\Z沈伟20220316.fif
[done]
Extracting EDF parameters from E:\jupyter file\EEG_process\patient_data\治疗脑电（第一批次）\Z沈伟20220318.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['File Create', 'Recording ends', 'Recording starts', 'SE', 'end', 'start']
Extracting EDF parameters from E:\jupyter file\EEG_process\patient_data\治疗脑电（第一批次）\Z沈伟20220318.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
['EEG Fp1-AV', 'EEG Fp2-AV', 'EEG F3-AV', 'EEG F4-AV', 'EEG C3-AV', 'EEG C4-AV', 'EEG P3-AV', 'EEG P4-AV', 'EEG O1-AV', 'EEG O2-AV', 'EEG T3-AV', 'EEG T4-AV', 'EEG T5-AV', 'EEG T6-AV', 'EEG A1-AV', 'EEG A2-AV', 'EEG Cz-AV', 'EEG ECG', 'EEG EMG']
Used Annotations descriptions: ['File Create', 'Recording ends', 'Recording starts', 'SE', 'end', 'start']
50914 67298
[49.72070312 49.72167969 49.72265625 ... 65.71777344 65.71875
 65.71972656]
Creating RawArr

<ipython-input-44-9e5188346da5>:73: RuntimeWarning: This filename (E:\jupyter file\EEG_process\SE_test\Z沈伟20220318.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_temp.save(path_name,overwrite=True)
<ipython-input-44-9e5188346da5>:73: RuntimeWarning: This filename (E:\jupyter file\EEG_process\SE_test\Z赵丽兰20220225.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_temp.save(path_name,overwrite=True)
<ipython-input-44-9e5188346da5>:73: RuntimeWarning: This filename (E:\jupyter file\EEG_process\SE_test\Z赵丽兰20220228.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_

Used Annotations descriptions: ['File Create', 'Recording ends', 'Recording starts', 'SE', 'end', 'start']
Extracting EDF parameters from E:\jupyter file\EEG_process\patient_data\治疗脑电（第一批次）\Z赵丽兰20220302.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
['EEG Fp1-AV', 'EEG Fp2-AV', 'EEG F3-AV', 'EEG F4-AV', 'EEG C3-AV', 'EEG C4-AV', 'EEG P3-AV', 'EEG P4-AV', 'EEG O1-AV', 'EEG O2-AV', 'EEG T3-AV', 'EEG T4-AV', 'EEG T5-AV', 'EEG T6-AV', 'EEG A1-AV', 'EEG A2-AV', 'EEG Cz-AV', 'EEG ECG', 'EEG EMG']
Used Annotations descriptions: ['File Create', 'Recording ends', 'Recording starts', 'SE', 'end', 'start']
4442 20826
[ 4.33789062  4.33886719  4.33984375 ... 20.33496094 20.3359375
 20.33691406]
Creating RawArray with float64 data, n_channels=15, n_times=16384
    Range : 0 ... 16383 =      0.000 ...    15.999 secs
Ready.
..\SE_test\Z赵丽兰20220302.fif
Writing E:\jupyter file\EEG_process\SE_test\Z赵丽兰20220302.fif
Closing E:\jupyter file\EEG_process\SE_test\Z赵丽

<ipython-input-44-9e5188346da5>:73: RuntimeWarning: This filename (E:\jupyter file\EEG_process\SE_test\Z赵丽兰20220302.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_temp.save(path_name,overwrite=True)
<ipython-input-44-9e5188346da5>:73: RuntimeWarning: This filename (E:\jupyter file\EEG_process\SE_test\Z赵丽兰20220304.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_temp.save(path_name,overwrite=True)


In [48]:
file_without_se

['../patient_data/治疗脑电（第一批次）\\Z刘家瑞20220304.edf',
 '../patient_data/治疗脑电（第一批次）\\Z张璐20220304.edf',
 '../patient_data/治疗脑电（第一批次）\\Z林攀攀20220221（未发作）.edf',
 '../patient_data/治疗脑电（第一批次）\\Z肖智华20220307（未发作）.edf']

In [49]:
df_parameter_train

,parameter
0,1
1,1
2,1
3,1
4,1
...,...
939,3
940,3
941,3
942,3


In [50]:
df_parameter_test

,parameter
0,3
1,2
2,1
3,4
4,3
5,1
6,4
7,2
8,4
9,1


In [51]:
train_output_path=("../SE_parameter/train_matched_stimulate_parameter.xlsx")
test_output_path=("../SE_parameter/test_matched_stimulate_parameter.xlsx")

In [52]:
df_parameter_train.to_excel(train_output_path,sheet_name="Sheet1")

In [53]:
df_parameter_test.to_excel(test_output_path,sheet_name="Sheet1")